# Métrica 3_2_05 — Donaciones o cesiones de inmuebles que deban reportarse como ingreso
| Escenario                                                                      | Condición                                            | Resultado        |
| :----------------------------------------------------------------------------- | :--------------------------------------------------- | :--------------- |
| No hay bienes inmuebles                                                        | —                                                    | ⚪ **SIN_DATO**   |
| Bien adquirido por compra o herencia                                           | `formaAdquisicion` distinta de “DONACIÓN” o “CESIÓN” | ⚪ **N/A**        |
| Bien con forma de adquisición “DONACIÓN” o “CESIÓN” pero sin ingreso reportado | `/ingresos/totalIngresosConclusionNetos/valor` ≤ 0   | 🔴 **NO_CUMPLE** |
| Bien con forma “DONACIÓN” o “CESIÓN” y con ingreso reportado                   | —                                                    | 🟢 **CUMPLE**    |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "3_2_05_DONACIONES_INMUEBLES_INGRESO"

DICT_INFO = {
    "bienes_path": "declaracion.situacionPatrimonial.bienesInmuebles.bienInmueble",
    "ingresos_path": "declaracion.situacionPatrimonial.ingresos.totalIngresosConclusionNetos.valor",
    "obligatorio": False
}

# --- Funciones auxiliares ---

def extraer_lista(doc, path):
    """Extrae una lista desde un path anidado."""
    try:
        partes = path.split(".")
        actuales = [doc]
        for p in partes:
            siguientes = []
            for actual in actuales:
                if isinstance(actual, dict):
                    valor = actual.get(p, None)
                    if valor is None:
                        continue
                    if isinstance(valor, list):
                        siguientes.extend(valor)
                    else:
                        siguientes.append(valor)
                elif isinstance(actual, list):
                    for sub in actual:
                        if isinstance(sub, dict):
                            valor = sub.get(p, None)
                            if valor is None:
                                continue
                            if isinstance(valor, list):
                                siguientes.extend(valor)
                            else:
                                siguientes.append(valor)
            actuales = siguientes
        return [a for a in actuales if isinstance(a, dict)]
    except Exception:
        return []


def extraer_valor(doc, path):
    """Extrae un valor numérico desde un path anidado."""
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, None)
            elif isinstance(actual, list) and len(actual) > 0:
                actual = actual[0].get(p, None)
            else:
                return 0
        return float(actual or 0)
    except Exception:
        return 0


# --- Evaluación principal ---

def evaluar_metrica(doc):
    bienes = extraer_lista(doc, DICT_INFO["bienes_path"])
    ingresos = extraer_valor(doc, DICT_INFO["ingresos_path"])

    if not bienes:
        return "SIN_DATO" if not DICT_INFO["obligatorio"] else "NO_CUMPLE"

    donacion = False
    for b in bienes:
        forma = str(b.get("formaAdquisicion", "")).upper()
        if any(p in forma for p in ["DONACIÓN", "CESIÓN"]):
            donacion = True
            break

    if not donacion:
        return "N/A"

    if ingresos <= 0:
        return "NO_CUMPLE"

    return "CUMPLE"


# --- Procesamiento MongoDB ---

def procesar_metrica_3_2_05():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0, "N/A": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            DICT_INFO["bienes_path"]: 1,
            DICT_INFO["ingresos_path"]: 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] = resultados.get(resultado, 0) + 1
            operaciones.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {METRIC_ID: resultado}},
                upsert=True
            ))

            if len(operaciones) >= 1000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_3_2_05()


Procesando métrica 3_2_05_DONACIONES_INMUEBLES_INGRESO...


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 documentos...
  > Procesados 28000 documentos...
  > Procesados 29000 document

KeyboardInterrupt: 